<a href="https://colab.research.google.com/github/chandanravic/ML/blob/main/AI_Village_Cluster_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
load the pretrained model
```



In [1]:
import requests

input_data = [0]

def query(input_data):
    response = requests.post('http://cluster1.advml.com/score', json={'data': input_data})
    return response.json()

#query(input_data)

install packages

In [2]:
!pip -q install skops
!pip -q install scikit-learn==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 21.2 MB/s eta 0:00:00


* import libraries

* load the files, ML file and the dataset to pandas dataframe, sorting the id and reset the index

* load the ML function pipe to predict the data, adding column pos_misclassified to check if prediction matches actual.
and adding 1 to each row just for fun i guess.

In [4]:
import skops.io as sio
import sklearn
import pandas as pd
import itertools

pipe = sio.load("/content/sample_data/census_model.skops", trusted=True)
data = pd.read_csv("/content/sample_data/census.csv").sort_values('id').reset_index(drop=True)

prediction = pipe.predict(data)
data['pos_missclassified'] =  prediction != data.income#(prediction == '>50K') & (data.income == '<=50K')
data['counter'] = 1

In [8]:
features = [ 'workclass', 'education.num', 'marital.status', 'occupation',
            'relationship', 'race', 'sex', 'native.country']



all_missclassifications = []
max_misclassifications = -1
max_misclassified_cluster = None
max_combination = None

for r in range(1, len(features) + 1):
    for subset in itertools.combinations(features, r):
        subset = list(subset)

        g = data.groupby(subset)[[ 'pos_missclassified',  'counter']].sum()
        group = g['pos_missclassified']/g['counter']
        grouped = group[g['counter'] >= 162]
        if len(grouped) == 0:
            continue

        current_max_misclassifications = grouped.max()
        all_missclassifications.append(grouped.values)
        if current_max_misclassifications > max_misclassifications:
            max_misclassifications = current_max_misclassifications
            max_misclassified_cluster = grouped.idxmax()
            max_combination = subset

            max_misclassified_cluster


max_misclassified_cluster


('Tech-support', 'Female')